In [ ]:
# -----------------------------------------------------------------------------------------------------------------------
# Documentation Block 
# -----------------------------------------------------------------------------------------------------------------------

#  ---------------- PROGRAM INFORMATION ---------------------------------------------------------------------------------
# Name: Product review data processing
# Author: Sufiyan Syed 
# Date: 09/22/2024

#  ---------------- PURPOSE ---------------------------------------------------------------------------------------------
# This program is designed to proccess product review data extracted from CSV files. It performs several key tasks:
# 1. Counts and displays the number of rows in each CSV file, as well as the total number of rows across all files.
# 2. Concatenates multiple CSV files into a single DataFrame.
# 3. Cleans the data by removing duplicates and unnecessary columns.
# 4. Translates non-English review titles and review texts to English using the Google Translate API.
# 5. Saves the cleaned and consolidated data into an Excel file for further analysis.

#  ---------------- SELECTS ---------------------------------------------------------------------------------------------
# The program selects:
# - CSV files that start with the prefix 'honeycomb' and end with '.csv' from the current working directory.
# - Specific columns from the concatenated DataFrame, such as 'Reviewer' and 'Review Text', to remove duplicates.
# - Review title and text for translation if the text is not already in English.

#  ---------------- NOTES -----------------------------------------------------------------------------------------------
# - The program assumes that review titles and texts may contain non-English characters and uses Google Translate to convert them to English.
# - The Google Translate API is used with the 'auto' source language detection. Ensure internet connectivity when running this section.
# - Make sure the CSV files have consistent column structures for concatenation to work correctly.
# - The final cleaned DataFrame is saved as an Excel file named 'velocityone_data50524.xlsx'.
# - The program includes basic checks to ensure duplicates are correctly removed and translations are performed where necessary.

#  ---------------- DEPENDENCIES --------------------------------------------------------------------------------
# Load Packages 
import pandas as pd  # pandas is used for reading CSV files into DataFrames, cleaning, manipulating the data, 
                     # and saving it to Excel or CSV files.

import openpyxl  # openpyxl is used to work with Excel files, enabling the program to save the cleaned data as an Excel file.

from googletrans import Translator  # Translator from googletrans is used to automatically detect and translate 
                                    # review text and titles to English from various languages.

import numpy as np  # numpy is used for efficient array operations and can handle missing values 
                    # (e.g., using NaN for missing data) in the DataFrame.

import string  # string provides utilities to work with strings, such as checking for punctuation, 
               # and manipulating text data (e.g., review content).

import traceback  # traceback is used to print detailed error messages and stack traces, helping to debug the program 
                  # by identifying the exact point where an error occurred.

import os  # os is used to interact with the operating system, such as getting the current working directory 
           # and listing files in a directory.

from datetime import datetime # get current date 

#  ---------------- ENVIORNMENT ----------------------------------------------------------------------------------
# Create program metadata 
filpath = os.getcwd()
prgname = os.path.splitext(os.path.basename(filpath))[0]
prgdate = datetime.now().strftime('%y%m%d')

In [ ]:
#  ---------------- DATA ---------------------------------------------------------------------------------------

# Output directories 
savdir1 = os.getcwd()

# Output files
savfil1 = os.path.join(savdir1, "productname_data_date.xlsx")

# Input directories & files
# -----------------------------------------------------------------------------------------------------------------------
# Inspect Row Counts for Product Review Data Files
# -----------------------------------------------------------------------------------------------------------------------
# Note that this program assumes csv files of review data are present in your current working directory. Alter the code
# under "Directory containing the CSV files" to specify the directory you want the program to look for your files

## Define Function to count rows
def count_rows(csv_file):
    df = pd.read_csv(csv_file)
    return len(df)

## Directory containing the CSV files
csv_directory = os.getcwd()

## List all CSV files in the directory
csv_files = [os.path.join(csv_directory, file) for file in os.listdir(csv_directory) if file.startswith('honeycomb') and file.endswith('.csv')]

## Dictionary to store counts for each file
file_counts = {}

## Variable to store the total number of rows
total_rows = 0

## Loop through CSV files and count rows
for file in csv_files:
    rows_count = count_rows(file)
    file_name = os.path.basename(file)
    file_counts[file_name] = rows_count
    total_rows += rows_count
    print(f"File: {file_name}, Rows: {rows_count}")

## Display the total number of rows
print(f"\nTotal number of rows across all files: {total_rows}")

## Clean enviornment
del csv_directory, file_counts, total_rows, file, file_name, rows_count 


File: honeycombxpc_tr_5s_p9.csv, Rows: 10
File: honeycombxpc_tr_1s_p1.csv, Rows: 10
File: honeycombxpc_other_1s_1.csv, Rows: 8
File: honeycombxpc_tr_5s_p8.csv, Rows: 10
File: honeycombxpc_tr_1s_p2.csv, Rows: 2
File: honeycombxpc_tr_5s_p10.csv, Rows: 10
File: honeycomb_otherreviews.csv, Rows: 32
File: honeycombxpc_other_3s_1.csv, Rows: 8
File: honeycombxpc_other2_3s_1.csv, Rows: 10
File: honeycombxpc_tr_2s_p1.csv, Rows: 4
File: honeycombxpc_other2_3s_2.csv, Rows: 6
File: honeycombxpc_tr_3s_p1.csv, Rows: 10
File: honeycombxpc_mr_5s_p10.csv, Rows: 10
File: honeycombxpc_other2_1s_2.csv, Rows: 1
File: honeycombxpc_tr_3s_p2.csv, Rows: 6
File: honeycombxpc_mr_5s_p9.csv, Rows: 10
File: honeycombxpc_other2_1s_1.csv, Rows: 10
File: honeycombxpc_mr_5s_p8.csv, Rows: 10
File: honeycombxpc_mr_5s_p5.csv, Rows: 10
File: honeycombxpc_mr_5s_p4.csv, Rows: 10
File: honeycombxpc_mr_5s_p6.csv, Rows: 10
File: honeycombxpc_mr_5s_p7.csv, Rows: 10
File: honeycombxpc_mr_5s_p3.csv, Rows: 10
File: honeycombxpc_oth

In [ ]:
# -----------------------------------------------------------------------------------------------------------------------
# Append Data Files
# -----------------------------------------------------------------------------------------------------------------------

## Initialize an empty list to store DataFrames
dfs = []

## Loop through CSV files and append them to the list of DataFrames
for file in csv_files:
    df = pd.read_csv(file, index_col=False)
    dfs.append(df)

## Concatenate all DataFrames into a single DataFrame
appended_df = pd.concat(dfs, ignore_index=True)

## Count rows to compare to individual totals
print("total rows:", appended_df.shape[0])

## Clean enviornment
del df, dfs, file, csv_files


total rows: 341


In [ ]:
# -----------------------------------------------------------------------------------------------------------------------
# Clean Data
# -----------------------------------------------------------------------------------------------------------------------

## Remove index
tmp = appended_df.drop(appended_df.columns[0], axis=1)
#tmp2 = tmp.drop(['Unamed'])

## Remove duplicates
colnames = tmp.columns.tolist()
print("Colnames:", colnames)

tmp2 = tmp.drop_duplicates(subset=['Reviewer', 'Review Text'], keep='first')
print("Filtered nrow", tmp2.shape[0])


### Checks

### Count duplicates
print("Number of Duplicates:", appended_df.duplicated(subset=['Reviewer', 'Review Text']).sum())

### Add count of filtered dataframe to count of duplicates. 
### If sum equals total rows of original dataframe, print "check". 
if tmp2.shape[0] + appended_df.duplicated(subset=['Reviewer', 'Review Text']).sum() == appended_df.shape[0]: print("Check")

### Count duplicates
tmp3 = tmp.duplicated(subset=['Reviewer', 'Review Text'], keep=False)
tmp4 = tmp[tmp3]
print("Number of Duplicates:", tmp4.shape[0]/2)

### Check if any duplicates are remaining
print("Remaining Duplicates:", tmp2.duplicated(subset=['Reviewer', 'Review Text']).sum())

## Clean enviornment 
tmp = tmp2

del tmp3, tmp4, tmp2, appended_df, colnames




Colnames: ['Reviewer', 'Star Rating', 'Review Title', 'Review Date', 'Purchase Verification', 'Review Text', 'Reviewer ', 'Review Site']
Filtered nrow 247
Number of Duplicates: 94
Check
Number of Duplicates: 94.0
Remaining Duplicates: 0


In [ ]:
## Clean review title
tmp['review_title'] = tmp['Review Title'].str.split('out of 5 stars', n=1).str[-1]

### Checks 
tmp2 = tmp[['review_title', 'Review Title']]

### Delete old column
tmp3 = tmp.drop(columns='Review Title')

## Drop review site column
tmp4 = tmp3.drop(columns='Review Site')

## Drop duplicate "Reviewer" column
tmp5 = tmp4.drop(tmp4.columns[5], axis=1)

# Clean evniornment
tmp = tmp5 

del tmp2, tmp3, tmp4, tmp5


In [ ]:
# -----------------------------------------------------------------------------------------------------------------------
# Translate any non-english reviews to english
# -----------------------------------------------------------------------------------------------------------------------

## Define translation function
def translate_to_english(text, src_language='auto'):
    try:
        if pd.isnull(text):
            return text
        elif all(char in string.punctuation or char.isspace() for char in text):
            print("Skipping punctuation-only text:", text)
            return text
        else: 
            translator = Translator()
            print("Translating:", text)
            translated_text = translator.translate(text, src=src_language, dest='en')
            print("Translating to English:", translated_text.text)
            return translated_text.text
    except Exception as e:
        print(f"An error occurred during translation: {e}")
        print(f"Original text: {text}")
        traceback.print_exc()  # Print the traceback for debugging
        return text

## Test function
text_to_translate = "Bonjour tout le monde"  # French text
translated_text = translate_to_english(text_to_translate)
print(translated_text)  # Output: Hello everyone

del text_to_translate, translated_text

Translating: Bonjour tout le monde
Translating to English: Hello everyone
Hello everyone


In [ ]:
## Translate review title and text
tmp2 = tmp

## Review titles
tmp2['review_title_oeng'] = tmp.apply(lambda row: translate_to_english(row['review_title']) 
                                 if row['review_title'] != 'en' 
                                 else row['review_title'], axis=1)

## Review text
tmp2['review_text_oeng'] = tmp.apply(lambda row: translate_to_english(row['Review Text']) 
                                 if row['Review Text'] != 'en'
                                 else row['Review Text'], axis=1)

In [ ]:
### Checks

### Spot check
tmp3 = tmp2[['Review Text', 'review_title', 'review_text_oeng', 'review_title_oeng']]

### View translated items
tmp4 = tmp3[(tmp3['Review Text'] != tmp3['review_text_oeng']) & (tmp3['review_title'] != tmp3['review_title_oeng'])]


In [ ]:
# -----------------------------------------------------------------------------------------------------------------------
# Clean and save
# -----------------------------------------------------------------------------------------------------------------------
del tmp3, tmp4, tmp 

## Clean names
tmp2.columns = tmp2.columns.str.strip().str.lower().str.replace(' ', '_' )

## Save the appended DataFrame to an Excel file
tmp2.to_excel(savfil1, index=False)
